#### GA算法和SA算法比较

In [187]:
class GA:
    from random import choice, choices, uniform
    # 编码方案(pop->code,code->pop)(格雷码)，适应度函数(pop->int)，初始种群(list)，种群大小(int)，选择算子(pops->pop)，交叉算子(pops->pop)\
        # 变异算子(pops->pop)，迭代次数(int)
    def __init__(self, coding = bin, evaluate= lambda x:int(x,2), \
        initial = list(range(23)), iterateDepth = 1000, endValue = 1) :
        
        self.population = initial # 种群初始化
        self.size = len(self.population) # 种群大小
        self.coding = coding # 编码函数
        self.eva = evaluate # 适应度 return int
        self.end = endValue # 优化值（默认优化函数为min）
        self.T = iterateDepth # 最大迭代深度
        # default
        self.Pv = 0.001 # 变异概率
        self.Pc = 0.5 # 交换概率
        self.Py = 1 - self.Pv - self.Pc # 复制概率
        
        
    # 参数设置
    def setP(self, Pv, Pc):  
        self.Pv = Pv  
        self.Pc = Pc  
        self.Py = 1 - Pv - Pc  
        return 1  
    
    def setEva(self, f):
        self.eva = lambda x:abs(f(x))

    
    # 选择复制函数
    def cho(self, pops):
        return self.choices(pops, [self.eva(pop) for pop in pops])[0]
    
    # 交换函数
    def cro(self,  pops):
        # 可以重样，考虑能自交吧
        father, mother = self.choices(pops,[self.eva(pop) for pop in pops], k = 2)
        k = self.choice( range( - min( len(father),len(mother) ) + 2 ,0) )
        son = father[:k] + mother[k:]
        return son
    
    # 突变函数
    def var(self, pops):
        pop  = self.choice(pops)
        if int(pop,2) > 0:
            k = self.choice(range(- len(pop) + 2, 0))
            newPop = pop[:k] + str(1-int(pop[k])) + pop[k+1:] if k != -1 \
                else pop[:k] + str(1-int(pop[k]))# 0b1的情况
        else:
            k = self.choice(range( -len(pop) + 3 , 0))
            newPop = pop[:k] + str(1-int(pop[k])) + pop[k+1:]
        return newPop
            
    # 最佳评估值
    def bestComfortPopValue(self, pops):
        return max([self.eva(pop) for pop in pops])
        
    # 主循环
    def main(self):
        depth = 0
        # 编码
        pops = [self.coding(pop) for pop in self.population]

        while self.bestComfortPopValue(pops) > self.end and depth < self.T:
            newPops = []
            for i in range(self.size):
                # 内循环，保证每次种群更新size次
                p = self.uniform(0,1)
                if p < self.Py:
                    newPop = self.cho(pops)
                elif p < self.Pc + self.Py:
                    newPop = self.cro(pops)
                else:
                    newPop = self.var(pops)
                newPops.append(newPop)

            depth += 1
            pops = newPops
        
        return  int(max(pops,key=self.eva),2)
        
        

In [188]:
a = GA()
def g(s):
    x = int(s,2)
    return  1000 - ( x **4 -3 * x**2 + 4* x-10)
a.setEva(g)
result = a.main()

In [189]:
print(result)

31
